<a href="https://colab.research.google.com/github/tgeral68/TP-1-chatbot/blob/main/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch

from matplotlib import pyplot as plt
from collections import Counter

In [2]:
dataset_path = 'atis_corpus/atis.{}.csv'

In [3]:
training_set = pd.read_csv(dataset_path.format('train'), index_col='id')
validation_set = pd.read_csv(dataset_path.format('dev'), index_col='id')
testing_set = pd.read_csv(dataset_path.format('test'), index_col='id')

FileNotFoundError: [Errno 2] No such file or directory: 'atis_corpus/atis.train.csv'

In [ ]:
training_set

In [ ]:
intent_index = {v: i for i, v in enumerate(np.unique(training_set['intent']))}
slot_name_index = {v: i for i, v in enumerate(np.unique(sum([slot.split() for slot in training_set['slots']], [])))}

number_intent = len(intent_index)
number_slot_name = len(slot_name_index)

print("There is {} possible intents and {} possible slots".format(number_intent, number_slot_name))

In [ ]:
class LabelEncoder:
    def __init__(self, index):
        self.index = index
        self.iindex = {v: k for k, v in index.items()}

    def get_id_list_from_text(self, text):
        return [self.index[w] for w in text.split()]

    def get_label_list_from_id(self, label_ids):
         return [self.iindex[lid] for lid in label_ids]

    def to_label_list(self, text):
        return text.split()

    def __str__(self):
        x = ""
        for k, v in self.iindex.items():
            x += "%s -> %s\n"%(v, k)
        return x
    # method call can be called by instance_object(parameters)
    def __call__(self, text):
        return self.get_id_list_from_text(text)

In [ ]:
intent_encoder = LabelEncoder(intent_index)
slot_encoder = LabelEncoder(slot_name_index)

slot_encoder(training_set.iloc[0]['slots'])
print(intent_encoder)

In [ ]:
'''This class should tokenize a text or a list of tokens.
'''
class Tokenizer:
    def __call__(self, data):
        # return the ids of the text or word sequence
        if(isinstance(data, str)):
            data = data.split()
        input_ids = []
        for w in data:
            try:
                input_ids.append(self.vocab_index[w])
            except:
                input_ids.append(self.vocab_index[self.unk_token])
        return input_ids

    def decode(self, data):
        decoded = []
        for dec in data:
            decoded.append(self.vocab[dec])
        return " ".join(decoded)

    def __len__(self):
        # get the vocabulary lenght
        return len(self.vocab)

class NaiveTokenizer(Tokenizer):
    def __init__(self, list_text, unk_token="unk"):
        # create a vocbulary index
        self.vocab = np.unique([word for text in list_text for word in text.split()] + ['unk'])
        self.vocab_index = {w: i for i, w in enumerate(self.vocab)}
        self.unk_token = unk_token

class SpacyTokenizer(Tokenizer):
    def __init__(self, spacy_tok):
        # create a vocbulary index
        self.vocab = [w for w in spacy_tok.vocab.strings]
        self.vocab_index = {w: i for i, w in enumerate(spacy_tok.vocab.strings)}
        self.unk_token = "unk"


In [ ]:
# create a simple tokenizer on all words of the corpus
naive_tokenizer = NaiveTokenizer(
    list(training_set['tokens']) +
    list(validation_set['tokens']) +
    list(testing_set['tokens'])
)
naive_tokenizer('BOS what is the meaning of meal codes EOS')

In [ ]:
import spacy
!python -m spacy download en_core_web_md

spacy_nlp = spacy.load('en_core_web_md')
spacy_tokenizer = SpacyTokenizer(spacy_nlp)
spacy_tokenizer('BOS what is the meaning of meal codes EOS')

In [ ]:
class BIONLUDataset():
    def __init__(self, data, intent_index, slot_index, text_transform):
        self.data = data
        self.i_enc = LabelEncoder(intent_index)
        self.s_enc = LabelEncoder(slot_index)
        self.t_transform = text_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['tokens']
        i_label = row['intent']
        s_label = row['slots']

        x = self.t_transform(text)
        i_id = self.i_enc(i_label)[0]
        s_ids = self.s_enc(s_label)

        assert(len(s_ids) == len(x))

        return {'text': text, 'i_label': i_label, 's_labels': s_label,
                'x': x, 'i_id': i_id, 's_ids': s_ids, "seq_len": len(x)}

In [ ]:
train = BIONLUDataset(training_set, intent_index, slot_name_index, naive_tokenizer)
val = BIONLUDataset(validation_set, intent_index, slot_name_index, naive_tokenizer)
test = BIONLUDataset(testing_set, intent_index, slot_name_index, naive_tokenizer)

In [4]:
import torch
from torch.nn.utils.rnn import pad_sequence

def rnn_embedding_slot_collator(batch):
    i_ids = []
    seq_len = []
    s_ids = []
    x = []
    mask = []
    for item in batch:
        i_ids.append(item['i_id'])
        s_ids.append(torch.LongTensor(item['s_ids']))
        seq_len.append(item['seq_len'])
        x.append(torch.Tensor(item['x']))
        mask.append(torch.ones(item['seq_len']))

    return{
        "i_ids" : torch.LongTensor(i_ids),
        "s_ids" : pad_sequence(s_ids, padding_value=-1, batch_first=True).long(),
        "input_ids" : pad_sequence(x, padding_value=0, batch_first=True).long(),
        "seq_len" : torch.LongTensor(seq_len),
        "mask" : pad_sequence(mask, padding_value=0, batch_first=True),
    }

In [5]:
from torch.utils.data import DataLoader

training_dl = DataLoader(train , batch_size=16,
                         collate_fn=rnn_embedding_slot_collator,
                         shuffle=True
                        )
validation_dl = DataLoader(val, batch_size=16,
                         collate_fn=rnn_embedding_slot_collator
                        )

NameError: name 'train' is not defined

In [6]:
next(iter(training_dl))

NameError: name 'training_dl' is not defined

In [28]:
from tqdm.notebook import trange, tqdm
from torch import optim
from torch import nn
from model import IntentRNN

class Trainer():
    def __init__(self, model):
        self.model = model
        self.optimizer = None

    def before_training_loop(self, learning_rate = 1e-3):
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criterion = nn.CrossEntropyLoss(ignore_index=-1)

    def validation_step(self, data):
        y_pred = self.model(data["input_ids"], data["seq_len"], data['mask'])
        y_intent_truth = data["i_ids"]
        y_slot_truth = data["s_ids"]
        #print(y_slot_truth)
        y_intent_pred = y_pred["y_intent"]
        #print(y_pred["y_slot"].shape)
        y_slot_pred = y_pred["y_slot"].view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1)
        minus_one = (data["mask"] - 1)
        y_slot_pred *= data["mask"].view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1)
        # y_slot_pred += minus_one.view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1)
        return {
            'mask' : minus_one.view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1),
            'len' : len(data['input_ids']),
            'intent_pred': y_intent_pred,
            'intent_truth': y_intent_truth,
            'slot_pred': y_slot_pred,
            'slot_truth': y_slot_truth
        }

    def training_step(self, data):
        y_pred = self.model(data["input_ids"], data["seq_len"], data['mask'])
        y_intent_truth = data["i_ids"]
        y_slot_truth = data["s_ids"]
        loss_intent = self.criterion(y_pred["y_intent"], y_intent_truth)

        y_slot_pred = y_pred["y_slot"].view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1)
        y_slot_pred *= data["mask"].view(y_slot_truth.shape[0] * y_slot_truth.shape[-1], -1)

        loss_slot = self.criterion(y_slot_pred, y_slot_truth.view(-1))
        (loss_slot + loss_intent).backward()
        return loss_slot.item() + loss_intent.item()

    def on_validation_end(self, resp):
        i_truth = [d["intent_truth"] for d in resp]
        s_truth = [d["slot_truth"].flatten() for d in resp]

        i_pred = [d["intent_pred"] for d in resp]
        s_pred =  [d["slot_pred"].argmax(-1).flatten() + d['mask'].max(-1).values.flatten() for d in resp]
        size = sum([d['len'] for d in resp ])
        print('Intent validation accuracy : %s'%(((torch.cat(i_pred).argmax(-1) == torch.cat(i_truth)).sum()/size).item(),))
        print('Slot validation accuracy : %s'%(((torch.cat(s_pred) == torch.cat(s_truth)).sum()/len(torch.cat(s_truth))).item(),))

    def validation(self, validation_dl):
        with torch.no_grad():
            resp = []
            for data in validation_dl:
                resp.append(self.validation_step(data))
        self.on_validation_end(resp)



    def fit(self,
            training_dl,
            validation_dl,
            learning_rate = 1e-3,
            validation_frequency = 128,
            max_iter = 10000,
            gpu=False
        ):
        if(gpu):
          self.model = self.model.cuda()
        self.before_training_loop(learning_rate)

        iter_count = 0
        loss_buffer = []
        pbar = trange(max_iter)

        while(True):
            for data in training_dl:
                if gpu:
                  data = {v.cuda() for k, v in data.items()}
                self.optimizer.zero_grad()
                loss_buffer += [self.training_step(data)]
                self.optimizer.step()

                if(iter_count  % validation_frequency == 0):
                    print("Loss at iteration %s is %s"%(iter_count, np.mean(loss_buffer)))
                    self.validation(validation_dl)

                iter_count += 1
                if(iter_count > max_iter):
                    return
                pbar.update(1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [30]:
import model
class MyModel(nn.Module):
    def __init__(self, embeddings):
        super().__init__()
        self.rnn = model.IntentSlotRNN(input_size=300, bidirectional=True)
        self.embeddings = embeddings

    def forward(self, input_ids, seq_len, mask=None):
        if(mask is None):
           mask =  input_ids.new_ones(input_ids.shape).float()
        emb = self.embeddings(input_ids)
        return self.rnn(emb, seq_len, mask)
initial_embeddings = nn.Embedding(len(naive_tokenizer), 300)
trainer = Trainer(MyModel(initial_embeddings))
trainer.fit(training_dl, validation_dl, validation_frequency=500, gpu=True)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx